In [ ]:
# Use BLIP to generate caption instead of fixed default_labels

import gradio as gr
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration
from deep_translator import GoogleTranslator
from gtts import gTTS
import tempfile
import torch

In [ ]:
# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load CLIP
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Load BLIP for caption generation
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# Languages
languages = {
    "English": "en", "Korean": "ko", "Japanese": "ja",
    "Chinese": "zh-CN", "Russian": "ru", "French": "fr",
    "Vietnamese": "vi", "Spanish": "es", "Portuguese": "pt"
}

In [ ]:
# Translate
def translate_word(word, lang_code):
    try:
        return GoogleTranslator(source="auto", target=lang_code).translate(word)
    except Exception as e:
        print("Translation error:", e)
        return "(แปลไม่ได้)"

# Text-to-speech
def speak_word(text, lang_code):
    try:
        tts = gTTS(text=text, lang=lang_code)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp:
            tts.save(tmp.name)
            return tmp.name
    except Exception as e:
        print("TTS error:", e)
        return None

# Predict using BLIP + translate + TTS
def predict(image, selected_langs):
    # ใช้ BLIP สร้างคำอธิบายภาพ (caption ภาษาอังกฤษ)
    inputs = blip_processor(image, return_tensors="pt").to(device)
    out = blip_model.generate(**inputs)
    eng_caption = blip_processor.decode(out[0], skip_special_tokens=True)

    # แปลเป็นไทยให้แสดงที่ช่อง prediction_text
    caption_thai = translate_word(eng_caption, "th")

    # เตรียมค่าที่จะส่งกลับ
    outputs = {"label": caption_thai}
    for lang in languages:
        if lang in selected_langs:
            translated = translate_word(eng_caption, languages[lang])
            audio_path = speak_word(translated, languages[lang])
            outputs[lang] = (translated, audio_path)
        else:
            outputs[lang] = ("", None)

    return [outputs["label"]] + [outputs[lang][0] for lang in languages] + [gr.update(value=outputs[lang][1], visible=bool(outputs[lang][1])) for lang in languages]

In [ ]:
# UI
with gr.Blocks() as app:
    gr.Markdown("# 📸 SnapTranslate By Pawit\nอัปโหลดภาพ → ทำนาย → เลือกภาษาที่ต้องการ → กดแปลภาษาและกดฟังเสียงภาษาที่ต้องการ")

    with gr.Row():
        image_input = gr.Image(type="pil", label="📤 อัปโหลดรูปภาพ")
        lang_select = gr.CheckboxGroup(choices=list(languages.keys()), label="🌐 เลือกภาษาที่ต้องการแปล")

    predict_btn = gr.Button("🧠 แปลภาพ")
    prediction_text = gr.Textbox(label="🔍 คำอธิบายจากภาพ")

    output_texts = {}
    audio_outputs = {}

    for lang in languages:
        with gr.Row():
            output_texts[lang] = gr.Textbox(label=f"{lang}:", visible=True)
            audio_outputs[lang] = gr.Audio(label="", visible=False)

    predict_btn.click(
        fn=predict,
        inputs=[image_input, lang_select],
        outputs=[prediction_text] + list(output_texts.values()) + list(audio_outputs.values())
    )

app.launch(debug=True, share=True)